<a href="https://colab.research.google.com/github/MatthewYancey/16-9GAN/blob/master/src/model_baseline_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# https://pytorch.org/tutorials/beginner/dcgan_faces_tutorial.html

from __future__ import print_function
import argparse
import os
import glob
import shutil
import random
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
from torch.utils.tensorboard import SummaryWriter

import torchvision

import torchvision.datasets as dset
from torch.utils.data import Dataset

import torchvision.transforms as transforms
import torchvision.utils as vutils

%matplotlib inline

In [3]:
# mounts the google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
# Set random seed for reproducibility
SEED = 0
random.seed(SEED)
torch.manual_seed(SEED)

# directories
ZIP_DIR = '/content/gdrive/My Drive/16:9GAN/data_out/frames_16_9.zip'
IMG_DIR = '/content/faces/'
LOG_DIR = '/content/gdrive/My Drive/16:9GAN/data_out/logs/model_001/'

# image properties
IMG_SIZE = 32
N_CHANNELS = 3

# graph
GEN_INPUT_SIZE = 100
N_LAYERS = int(np.log(IMG_SIZE) / np.log(2)) - 1
N_GEN_CHANNELS = 128
N_DISC_CHANNELS = 128
beta1 = 0.5

# training
N_EPOCHS = 100
LR = 0.0002
N_WORKERS = 2
BATCH_SIZE = 16
ngpu = 1

# checkpoints and logs
CHECKPOINT_TYPE = 'none' # prev_model will load the previous model, prev_checkpoint will load the last checkpoint, none will do none. 

print('Number of layers: ' + str(N_LAYERS))

Number of layers: 4


In [13]:
ZIP_DIR = '/content/gdrive/My Drive/16:9GAN/data_out/anime-faces.tar.gz'
IMG_DIR = '/content/frames/'
shutil.unpack_archive(ZIP_DIR, IMG_DIR, 'gztar')

FileNotFoundError: ignored